In [4]:
import pandas as pd
import numpy as np
from datetime import datetime

In [106]:
from sklearn.preprocessing import MinMaxScaler

In [83]:
import tensorflow as tf
from tensorflow.keras.models import load_model

In [72]:
df = pd.read_csv('CCD_test.csv', index_col=0)
df

,scenario_set,DATE (MM/DD),MST,Global CMP22 (vent/cor) [W/m^2],Direct sNIP [W/m^2],Azimuth Angle [degrees],Tower Dry Bulb Temp [deg C],Tower Wet Bulb Temp [deg C],Tower Dew Point Temp [deg C],Tower RH [%],Total Cloud Cover [%],Peak Wind Speed @ 6ft [m/s],Avg Wind Direction @ 6ft [deg from N],Station Pressure [mBar],Precipitation (Accumulated) [mm],Snow Depth [cm],Moisture,Albedo (CMP11)
0,0,1-Dec,7:50,54.3644,464.075,122.61168,-0.7910,-5.3070,-15.6070,27.230,3.0,0.95,243.70,810.312,0.0,0.080,0.0,0.4190
1,0,1-Dec,7:51,56.6827,472.765,122.77874,-0.6920,-5.1580,-15.6580,26.880,2.0,0.00,0.00,810.301,0.0,0.051,0.0,0.4066
2,0,1-Dec,7:52,57.8298,477.189,122.86244,-0.6315,-5.1465,-15.6465,26.775,2.0,0.00,0.00,810.295,0.0,0.034,0.0,0.3995
3,0,1-Dec,7:53,58.9769,481.613,122.94614,-0.5710,-5.1350,-15.6350,26.670,2.0,0.00,0.00,810.289,0.0,0.017,0.0,0.3924
4,0,1-Dec,7:54,61.2684,489.245,123.11386,-0.4860,-5.1240,-15.6240,26.510,2.0,0.00,0.00,810.307,0.0,-0.003,0.0,0.3778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55795,154,Dec-31,14:25,358.4020,338.925,207.53615,5.9950,-0.0470,-8.9470,30.470,54.0,1.45,18.52,811.496,0.0,2.779,0.0,0.3348
55796,154,Dec-31,14:26,375.2150,445.022,207.76500,5.9300,-0.1990,-9.4990,29.150,51.0,1.45,16.65,811.486,0.0,2.809,0.0,0.3322
55797,154,Dec-31,14:27,422.8910,645.908,207.99338,5.8570,-0.2990,-9.5990,29.040,48.0,1.20,12.84,811.481,0.0,2.825,0.0,0.3428
55798,154,Dec-31,14:28,476.0710,831.800,208.22132,5.8090,-0.1460,-8.9460,30.870,47.0,1.20,11.94,811.462,0.0,2.826,0.0,0.3337


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55800 entries, 0 to 55799
Data columns (total 18 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   scenario_set                           55800 non-null  int64  
 1   DATE (MM/DD)                           55800 non-null  object 
 2   MST                                    55800 non-null  object 
 3   Global CMP22 (vent/cor) [W/m^2]        55800 non-null  float64
 4   Direct sNIP [W/m^2]                    55800 non-null  float64
 5   Azimuth Angle [degrees]                55800 non-null  float64
 6   Tower Dry Bulb Temp [deg C]            55800 non-null  float64
 7   Tower Wet Bulb Temp [deg C]            55800 non-null  float64
 8   Tower Dew Point Temp [deg C]           55800 non-null  float64
 9   Tower RH [%]                           55800 non-null  float64
 10  Total Cloud Cover [%]                  55800 non-null  float64
 11  Pe

In [74]:
def get_wind_direction(degrees):
    if degrees >= 337.5 or degrees < 22.5:
        return 'North'
    elif 22.5 <= degrees < 67.5:
        return 'North-East'
    elif 67.5 <= degrees < 112.5:
        return 'East'
    elif 112.5 <= degrees < 157.5:
        return 'South-East'
    elif 157.5 <= degrees < 202.5:
        return 'South'
    elif 202.5 <= degrees < 247.5:
        return 'South-West'
    elif 247.5 <= degrees < 292.5:
        return 'West'
    elif 292.5 <= degrees < 337.5:
        return 'North-West'

In [75]:
df['Wind Direction'] = df['Avg Wind Direction @ 6ft [deg from N]'].apply(get_wind_direction)

df['Wind Direction'] = df['Wind Direction'].map({'North': 0, 'North-East': 1, 'East': 2, 'South-East': 3, 'South': 4, 'South-West': 5, 'West': 6, 'North-West': 7})

In [76]:
ser = pd.Series()

for i in df['scenario_set'].unique():
    group = df.groupby('scenario_set').get_group(i)['Peak Wind Speed @ 6ft [m/s]']
    arr = group.rolling(window=5).mean()

    for i in range(5):
        arr.iloc[i] = group.iloc[:i+1].mean()

    ser = pd.concat([ser, arr])

df['Avg. Wind Speed'] = ser

C:\Users\vansh\AppData\Local\Temp\ipykernel_1876\3887619049.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ser = pd.Series()


In [77]:
ser = pd.Series()

for i in range(155):
    group = df.groupby('scenario_set').get_group(i)['Tower Dew Point Temp [deg C]']
    arr = group.rolling(window=60).mean()

    for i in range(60):
        arr.iloc[i] = group.iloc[:i+1].mean()

    ser = pd.concat([ser, arr])

df['Avg. Tower Temp'] = ser

C:\Users\vansh\AppData\Local\Temp\ipykernel_1876\1616234150.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ser = pd.Series()


In [78]:
df['Humidity_Temp_Interaction'] = df['Tower RH [%]'] * df['Tower Dry Bulb Temp [deg C]']

In [79]:
R = 287.058
df['Air_Density'] = (df['Station Pressure [mBar]'] * 100) / (R * (df['Tower Dry Bulb Temp [deg C]'] + 273.15))

In [80]:
df['Saturation_Vapor_Pressure'] = 6.11 * 10 ** ((7.5 * df['Tower Dry Bulb Temp [deg C]']) / (237.3 + df['Tower Dry Bulb Temp [deg C]']))

In [81]:
df['Vapor_Pressure'] = (df['Tower RH [%]'] / 100) * df['Saturation_Vapor_Pressure']

In [82]:
cols = ['DATE (MM/DD)', 'MST', 'scenario_set']

col_train = df.drop(cols, axis=1).columns.tolist()
col_train

['Global CMP22 (vent/cor) [W/m^2]',
 'Direct sNIP [W/m^2]',
 'Azimuth Angle [degrees]',
 'Tower Dry Bulb Temp [deg C]',
 'Tower Wet Bulb Temp [deg C]',
 'Tower Dew Point Temp [deg C]',
 'Tower RH [%]',
 'Total Cloud Cover [%]',
 'Peak Wind Speed @ 6ft [m/s]',
 'Avg Wind Direction @ 6ft [deg from N]',
 'Station Pressure [mBar]',
 'Precipitation (Accumulated) [mm]',
 'Snow Depth [cm]',
 'Moisture',
 'Albedo (CMP11)',
 'Wind Direction',
 'Avg. Wind Speed',
 'Avg. Tower Temp',
 'Humidity_Temp_Interaction',
 'Air_Density',
 'Saturation_Vapor_Pressure',
 'Vapor_Pressure']

In [107]:
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[col_train])

In [112]:
x = []

for i in range(0, len(scaled_features), 360):
    x.append(scaled_features[240+i:360+i, :])

x = np.asarray(x)
x.shape

(155, 120, 22)

In [84]:
gru_lstm = load_model('models/gru_lstm_v3.h5', compile=False)

In [85]:
tdnn = load_model('models/tdnn_v3.h5', compile=False)

In [86]:
transformer = load_model('models/transformer_v2.keras', compile=False)

In [113]:
gru_lstm_pred = gru_lstm.predict(x)

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 171ms/step


In [114]:
tdnn_pred = tdnn.predict(x)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


In [115]:
transformer_pred = transformer.predict(x)

4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/stepWARNING:tensorflow:5 out of the last 11 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000002275C627010> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step


In [116]:
preds = (0.5*gru_lstm_pred + 0.3*tdnn_pred + 0.2*transformer_pred)

In [117]:
preds.shape

(155, 3)

In [118]:
out = pd.DataFrame({'scenario_set': df['scenario_set'].unique(), '15': preds[:, 0], '25': preds[:, 1], '30': preds[:, 2]})

In [120]:
out.to_csv('submission.csv', index=False)